In [1]:
#data prefix for output data
yearmonth = "202405"

In [2]:
#import necessary packages
import platform
import os
import sys
import time
#for managing the data
import pandas as pd

#install !pip install webdriver-manager
#webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pathlib

import os
from io import StringIO

from tqdm import tqdm

In [3]:
#get current path
curr_path = os.getcwd()
project_tag = curr_path.split("/")[-1]
grouping = curr_path.split("/")[-2]
output_path = "../../../data/{}/{}/{}/".format(grouping,project_tag,yearmonth )

#create data output path if it doesn't exist
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) 

In [4]:
#load in level 2 data
df = pd.read_csv(output_path + "2_level_table.csv")
print(df.shape)

(15570, 8)


In [5]:
#open web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [6]:
#place holder for results
result_list = []

In [8]:
for i,r in tqdm( df.iterrows() ):
    #resume point in case it crashes
    last_i = len(result_list)-1
    if i<last_i:
        continue
        
    url_inspection = r['url_inspection_full']

    temp_result = {}
    
    #load the main landing page
    driver.get(url_inspection)
    
    try:
        #find disclaimer button
        disclamer_btn = driver.find_element(by='name', value="AcceptDisclaimerButton")
    
        #click the disclaimer button to move to the data page
        disclamer_btn.click()
    except:
        pass
    
    #get inspection text
    tables = driver.find_elements(By.CSS_SELECTOR, "article")
    
    #gather resulting texts
    temp_result['url_inspection_full'] = url_inspection
    temp_result['inspection_results'] =  tables[-2].text
    temp_result['actions_taken'] = tables[-1].text
    
    result_list.append(temp_result)
 

15570it [51:33,  5.03it/s]  


In [9]:
final_df = pd.DataFrame(result_list)
print(final_df.shape)

(15571, 3)


In [10]:
display(final_df.head())

,url_inspection_full,inspection_results,actions_taken
0,https://inspections.healthunit.com/Portal/Insp...,Equipment & Utensils Sanitation\nUtensils are ...,Food Handler Education On-site\nCorrection(s) ...
1,https://inspections.healthunit.com/Portal/Insp...,No results to show,Satisfactory - No Action Required
2,https://inspections.healthunit.com/Portal/Insp...,Equipment & Utensils Sanitation\nMechanical di...,Food Handler Education On-site\nCorrection(s) ...
3,https://inspections.healthunit.com/Portal/Insp...,Food Handler Training\nFood handler or supervi...,Food Handler Education On-site\nCorrection(s) ...
4,https://inspections.healthunit.com/Portal/Insp...,"Sanitary Maintenance & Construction\nRooms, wa...",Correction(s) Required


In [11]:
final_df.to_csv( output_path + "3_level_table.csv", index=False)

In [12]:
#close webdriver
try:
    driver.close()
except:
    pass